In [17]:
import sqlite3
import pandas as pd
from pyhwpx import Hwp
import os
import re

#region 공통 데이터
#안전관리계획서1의 데이터 딕셔너리(엑셀의 열 / 데이터의 종류)
안전계획서1_dic ={
    "파일이름" : "",
    "현장명" : "",
    "현장소재지" : "",
    "공사금액" : 0.0,
    "보고서_날짜_년" : "",
    "보고서_날짜_월" : "",
    "시공자" : "",
    "설계자" : "",
    "공사개요_대상공사" : "",
    "공사개요_구조" : "",
    "공사개요_개소" : 0,
    "공사개요_층수지하" : 0,
    "공사개요_층수지상" : 0,
    "공사개요_굴착깊이" : 0.0,
    "공사개요_최고높이" : 0.0,
    "공사개요_연면적" : 0.0,
    "기타특수구조물개요" : "",
    "주요공법1" : "",
    "주요공법2" : "",
    "주요공법3" : "",
    "주요공법4" : "",
    "주요공법5" : "",
    "주요공법6" : "",
    "주요공법7" : "",
    "주요공법8" : "",
    "주요공법9" : "",
    "주요공법10" : "",
    "파일경로" : "",
    "비고" : "",
}

cursor_positions = []  # 문단 앞 커서 위치를 담는 리스트 (list, para, pos)
section_titles = []    # 각 위치에 대응하는 목차 또는 섹션 이름 **목차명=필드명이되도록할것
정렬된_cursor_positions=[]
정렬된_section_titles=[]
custom_data = []

def 안전관리계획서1_데이터_초기화():
    """
    공통 데이터를 초기화합니다.
    """
    for key in 안전계획서1_dic.keys():
        안전계획서1_dic[key] = "" if isinstance(안전계획서1_dic[key], str) else 0 if isinstance(안전계획서1_dic[key], int) else 0.0

def 커서_섹션명_커스텀_리스트_데이터_초기화():
    global cursor_positions
    global section_titles
    global custom_data 
    cursor_positions = []  # 문단 앞 커서 위치를 담는 리스트 (list, para, pos)
    section_titles = []    # 각 위치에 대응하는 목차 또는 섹션 이름 **목차명=필드명이되도록할것
    custom_data = []

#endregion

#################################################################



In [18]:
#region 데이터베이스 초기화 및 연결 함수
def connect_db(db_name='data.db'):
    """
    SQLite 데이터베이스에 연결합니다.
    데이터베이스 파일이 없으면 자동으로 생성됩니다.

    :param db_name: 데이터베이스 파일 이름 (기본값: 'data.db')
    :return: 데이터베이스 연결 객체와 커서
    """
    if not db_name.endswith(".db"):
        raise ValueError("데이터베이스 파일 이름은 .db 확장자를 가져야 합니다.")
    
    # 데이터베이스 연결 (파일이 없으면 자동으로 생성됨)
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    return conn, cursor



In [19]:
# 테이블 생성 함수
def initialize_db(db_name, table_name, data_dict):
    """
    딕셔너리 키를 기반으로 데이터베이스 테이블을 생성합니다.
    """
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    # 딕셔너리에서 열 생성 쿼리 동적 작성
    columns = ', '.join([f"{key} {get_sqlite_type(value)}" for key, value in data_dict.items()])
    fixed_columns = "섹션명 TEXT, 리스트 INTEGER, para INTEGER, pos INTEGER"
    query = f"CREATE TABLE IF NOT EXISTS {table_name} (id INTEGER PRIMARY KEY AUTOINCREMENT, {columns}, {fixed_columns})"
    
    cursor.execute(query)
    conn.commit()
    conn.close()

def get_sqlite_type(value):
    """
    딕셔너리 값의 데이터 타입에 따라 SQLite 데이터 타입 반환.
    """
    if isinstance(value, int):
        return "INTEGER"
    elif isinstance(value, float):
        return "REAL"
    elif isinstance(value, str):
        return "TEXT"
    else:
        return "TEXT"  # 기본값



In [20]:
# 데이터 삽입 함수 / 문서 전체를 한방에 db화함.
def insert_data(
    db_name, table_name, data_dict, 섹션명_리스트= 정렬된_section_titles, 데이터_리스트=정렬된_cursor_positions
):
    """
    딕셔너리와 리스트를 사용해 데이터베이스에 데이터를 삽입합니다.
    
    :param db_name: 데이터베이스 파일 이름
    :param table_name: 테이블 이름
    :param data_dict: 삽입할 공통 데이터 딕셔너리
    :param 섹션명_리스트: 섹션명을 담은 리스트
    :param 데이터_리스트: (리스트, para, pos) 튜플 리스트
    """
    if len(섹션명_리스트) != len(데이터_리스트):
        raise ValueError("섹션명_리스트와 데이터_리스트의 길이가 다릅니다.")
    
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # 공통 데이터 열과 값
    common_columns = ', '.join(data_dict.keys())
    common_values = tuple(data_dict.values())

    # 섹션명_리스트와 데이터_리스트를 조합하여 삽입
    for 섹션명, (리스트, para, pos) in zip(섹션명_리스트, 데이터_리스트):
        query = f'''
            INSERT INTO {table_name} (
                {common_columns}, 섹션명, 리스트, para, pos
            ) VALUES ({', '.join(['?'] * (len(data_dict) + 3))})
        '''
        values = common_values + (섹션명, 리스트, para, pos)
        cursor.execute(query, values)

    conn.commit()
    conn.close()
    print(f"{table_name} 테이블에 데이터가 삽입되었습니다.")



In [21]:
# 데이터 조회 함수 # todo 
def fetch_data(db_name, table_name, condition=None):
    """
    데이터 조회 함수. 조건에 맞는 데이터를 반환.
    
    :param db_name: 데이터베이스 파일 이름
    :param table_name: 테이블 이름
    :param condition: 조건 (예: "id = 1")
    :return: 조회된 데이터 리스트
    """
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    query = f"SELECT * FROM {table_name}"
    if condition:
        query += f" WHERE {condition}"
    
    cursor.execute(query)
    rows = cursor.fetchall()
    conn.close()
    return rows



In [22]:
#데이터 조회2 ##이건 좀더 고민해야함 todo
def fetch_data_with_fallback(column=None, keyword=None):
    """특정 열 검색 / 모든 열 검색 / 조건 없이 조회"""
    conn, cursor = connect_db()
    if column and keyword:  # 특정 열에 대한 검색
        query = f'SELECT * FROM file_data WHERE {column} LIKE ?'
        cursor.execute(query, (f'%{keyword}%',))
    elif keyword:  # 모든 열에서 검색
        query = '''
            SELECT * FROM file_data
            WHERE 파일이름 LIKE ? OR 현장명 LIKE ? OR 현장소재지 LIKE ? OR 공사개요_대상공사 LIKE ?
        '''
        cursor.execute(query, (f'%{keyword}%',) * 4)
    else:  # 조건 없이 전체 조회
        query = 'SELECT * FROM file_data'
        cursor.execute(query)

    rows = cursor.fetchall()
    conn.close()
    return rows



In [23]:
#custom_data로 추출하기
def fetch_custom_data():
    """
    데이터베이스에서 데이터를 가져와 custom_data 형식으로 반환합니다.
    """
    conn, cursor = connect_db()
    
    # 필요한 데이터만 SELECT (리스트, para, pos와 섹션명만 추출)
    cursor.execute('SELECT 리스트, para, pos, 섹션명 FROM file_data ORDER BY id')
    rows = cursor.fetchall()  # 데이터베이스에서 행 가져오기
    conn.close()
    
    # 데이터를 custom_data 형식으로 변환
    custom_data = [((row[0], row[1], row[2]), row[3]) for row in rows]
    return custom_data

#custom_data로 특정id만 추출하기(id의 위치),(id+1의 위치)(id의 섹션명)
def fetch_custom_data_with_next_positions(id_list):
    """
    주어진 ID 리스트에 해당하는 데이터를 custom_data 형식으로 반환.
    (id의 시작 좌표), (id+1의 시작 좌표), "id 섹션명" 구조로 변환.
    """
    conn, cursor = connect_db()
    
    # ID 리스트를 조건으로 데이터 가져오기
    placeholders = ', '.join('?' for _ in id_list)
    query = f'''
        SELECT id, 리스트, para, pos, 섹션명 
        FROM file_data 
        WHERE id IN ({placeholders}) 
        ORDER BY id
    '''
    cursor.execute(query, id_list)
    rows = cursor.fetchall()
 

     # ID의 다음 위치를 가져오기 위한 사전 조회
    next_positions = {}
    for id_ in id_list:
        cursor.execute('''
            SELECT 리스트, para, pos 
            FROM file_data 
            WHERE id = ?
        ''', (id_ + 1,))
        next_row = cursor.fetchone()
        if next_row:
            next_positions[id_] = next_row
        else:
            next_positions[id_] = "문서끝"

    # 데이터 변환
    custom_data = []
    for row in rows:
        current_id, 리스트, para, pos, 섹션명 = row
        next_position = next_positions.get(current_id, "문서끝")
        if next_position != "문서끝":
            custom_data.append(
                ((리스트, para, pos), (next_position[0], next_position[1], next_position[2]), 섹션명)
            )
        else:
            custom_data.append(
                ((리스트, para, pos), "문서끝", 섹션명)
            )
    conn.close()
    return custom_data

# 데이터 삭제 함수
def delete_data(row_id):
    conn, cursor = connect_db()
    cursor.execute('DELETE FROM file_data WHERE id = ?', (row_id,))
    conn.commit()
    conn.close()

# 데이터 업데이트 함수  ##이것도 잘모르겠음. todo
def update_data(row_id, **kwargs):
    """
    특정 ID의 데이터를 업데이트합니다.
    kwargs: 업데이트할 열 이름과 값의 딕셔너리 (예: {"파일이름": "새 파일명", "현장명": "새 현장명"})
    """
    conn, cursor = connect_db()
    updates = []
    values = []

    for key, value in kwargs.items():
        updates.append(f"{key} = ?")
        values.append(value)
    
    values.append(row_id)
    query = f'UPDATE file_data SET {", ".join(updates)} WHERE id = ?'
    cursor.execute(query, values)
    conn.commit()
    conn.close()



In [24]:
# 데이터 출력 함수
def print_data(db_name, table_name, condition=None):
    """
    데이터베이스에서 데이터를 조회하고 출력합니다.

    :param db_name: 데이터베이스 파일 이름
    :param table_name: 테이블 이름
    :param condition: SQL 조건문 (예: "id = 1")
    """
    rows = fetch_data(db_name, table_name, condition)
    
    if not rows:
        print("데이터가 없습니다.")
        return

    # 결과 출력
    for row in rows:
        print(row)



In [25]:
def reset_table():
    """
    테이블을 삭제하고 다시 생성하여 ID를 포함한 모든 데이터를 초기화합니다.
    """
    conn, cursor = connect_db()

    # 테이블 삭제
    cursor.execute('DROP TABLE IF EXISTS file_data')
    conn.commit()

    # 테이블 재생성
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS file_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            파일이름 TEXT,
            현장명 TEXT,
            현장소재지 TEXT,
            공사금액 REAL,
            보고서_날짜_년 TEXT,
            보고서_날짜_월 TEXT,
            시공자 TEXT,
            설계자 TEXT,
            공사개요_대상공사 TEXT,
            공사개요_구조 TEXT,
            공사개요_개소 INTEGER,
            공사개요_층수지하 INTEGER,
            공사개요_층수지상 INTEGER,
            공사개요_굴착깊이 REAL,
            공사개요_최고높이 REAL,
            공사개요_연면적 REAL,
            기타특수구조물개요 TEXT,
            주요공법1 TEXT,
            주요공법2 TEXT,
            주요공법3 TEXT,
            주요공법4 TEXT,
            주요공법5 TEXT,
            주요공법6 TEXT,
            주요공법7 TEXT,
            주요공법8 TEXT,
            주요공법9 TEXT,
            주요공법10 TEXT,
            파일경로 TEXT,
            섹션명 TEXT,
            리스트 INTEGER,
            para INTEGER,
            pos INTEGER,
            비고 TEXT
        )
    ''')
    conn.commit()
    conn.close()
    print("테이블이 초기화되었습니다.")
    
# 데이터베이스 초기화
initialize_db(db_name='nnn_1',table_name='con_1',data_dict=안전계획서1_dic)
print_data('nnn_1', 'con_1')
#endregion

#################################################################



데이터가 없습니다.


In [27]:
#region hwp 초기화

hwp = Hwp(new=True)
#hwp2 = Hwp(new=True)

hwp.Open(r"C:\Users\codek\Desktop\건설자료\01. [변경] 안전관리계획서 제1편 - 천안 성성5지구 IPARK 신축공사(R4) 정기안전점검 수정 240925.hwp")
#hwp2.Open(r"C:\Users\codek\Documents\GitHub\connwnw\test\샘플_템플릿.hwp")

hwp.MoveDocBegin()

def 오른_표이동(n):
    """n번 오른쪽으로 이동"""
    for _ in range(n):
        hwp.TableRightCell()
#캐럿 포지션 저장
def 위치추가():
    #목차 문단 맨 앞의 위치 추가
    cursor_positions.append(hwp.GetPos())

    #목차 문단 셀선택 후 추가하고 다시 첫 위치로 돌아가기
    hwp.MoveSelParaEnd()
    section_titles.append(hwp.get_selected_text())
    hwp.MoveParaBegin()
#마지막 처음 추가하기
def 처음지점추가():
    hwp.MoveDocBegin()
    cursor_positions.append(hwp.GetPos())
    section_titles.append("문서처음")
def 문단시작지점추가():
        문단시작_list =[]
        for i in hwp.ctrl_list:
            if i.UserDesc == "새 번호":#컨트롤이 표일 경우
                문단시작_list.append(i)#리스트에 저장
        for i in range(len(문단시작_list)):
            hwp.move_to_ctrl(문단시작_list[i])
            hwp.MoveLeft()
            cursor_positions.append(hwp.GetPos())
            section_titles.append(f"문단{i+1}")
#마지막 지점 추가하기
def 마지막지점추가():
    hwp.MoveDocEnd()
    cursor_positions.append(hwp.GetPos())
    section_titles.append("문서끝")
#리스트 para기준 재정렬
def 재정렬_파라기준(cursor_positions, section_titles):
    """
    cursor_positions의 'para' 값을 기준으로 cursor_positions와 section_titles를 재정렬하는 함수.
    
    매개변수:
    - cursor_positions (list of tuples): (table, para, dotPos) 형식의 커서 위치 리스트.
    - section_titles (list of str): 해당 커서 위치에 대응하는 섹션 제목 리스트.
    
    반환값:
    - tuple: 정렬된 cursor_positions와 section_titles.
    """
    # cursor_positions와 section_titles를 결합
    결합_데이터 = list(zip(cursor_positions, section_titles))
    
    # 'para' 값(튜플의 두 번째 값)을 기준으로 정렬
    정렬된_데이터 = sorted(결합_데이터, key=lambda x: x[0][1])
    
    # 정렬된 데이터를 다시 두 개의 리스트로 분리
    정렬된_cursor_positions, 정렬된_section_titles = zip(*정렬된_데이터)
    
    return list(정렬된_cursor_positions), list(정렬된_section_titles)

#endregion

#################################################################

#region 문서 공통 info 추출
#파일경로 추출
안전계획서1_dic["파일경로"] = hwp.Path

#파일이름 추출
안전계획서1_dic["파일이름"] = os.path.basename(안전계획서1_dic["파일경로"])
print(안전계획서1_dic["파일경로"], 안전계획서1_dic["파일이름"])

##처음 위치로##
hwp.Cancel()
hwp.MoveDocBegin()

# 정규표현식으로 연도와 월 추출
연월텍스트 = hwp.GetPageText()
패턴 = r'(\d{4})\.\s*(\d{2})'
매칭 = re.search(패턴, 연월텍스트)

if 매칭:
    안전계획서1_dic["보고서_날짜_년"], 안전계획서1_dic["보고서_날짜_월"] = 매칭.groups()
    #print(f"연도: {안전계획서1_dic["보고서_날짜_년"]}")
    #print(f"월: {안전계획서1_dic["보고서_날짜_월"]}")
else:
    print("연도와 월을 찾을 수 없습니다.")

#region###--공사개요서 뒤지기--###
if hwp.find_forward("1. 공사 개요서"):
    hwp.SelectCtrlFront()
    hwp.ShapeObjTableSelCell() # 첫번째 셀 선택
else : print("공사개요서 못찾음.")

#현장명
오른_표이동(2)
안전계획서1_dic["현장명"] = hwp.get_selected_text()

#소재지
오른_표이동(2)
안전계획서1_dic["현장소재지"] = hwp.get_selected_text()

#공사금액
오른_표이동(4)
안전계획서1_dic["공사금액"] = hwp.get_selected_text()

#시공자회사명
오른_표이동(3)
안전계획서1_dic["시공자"] = hwp.get_selected_text()

#설계자 회사명
오른_표이동(25)
안전계획서1_dic["설계자"] = hwp.get_selected_text()

#대상공사
오른_표이동(41)
안전계획서1_dic["공사개요_대상공사"] = hwp.get_selected_text()

#구조
오른_표이동(1)
안전계획서1_dic["공사개요_구조"] = hwp.get_selected_text()

#개소
오른_표이동(1)
안전계획서1_dic["공사개요_개소"] = hwp.get_selected_text()

오른_표이동(1)
안전계획서1_dic["공사개요_층수지하"] = hwp.get_selected_text()

오른_표이동(1)
안전계획서1_dic["공사개요_층수지상"] = hwp.get_selected_text()

오른_표이동(1)
안전계획서1_dic["공사개요_굴착깊이"] = hwp.get_selected_text()

오른_표이동(1)
안전계획서1_dic["공사개요_최고높이"] = hwp.get_selected_text()

오른_표이동(1)
안전계획서1_dic["공사개요_연면적"] = hwp.get_selected_text()

오른_표이동(2)
안전계획서1_dic["기타특수구조물개요"] = hwp.get_selected_text()

#공법리스트
오른_표이동(2)
주요공법 = hwp.get_selected_text()  # 텍스트 가져오기
공법리스트 = [공법.strip() for 공법 in 주요공법.split('\n') if 공법.strip()]  # 줄 단위로 나누고 양쪽 공백 제거 및 빈 줄 제거

# 공법리스트 저장
안전계획서1_dic["주요공법리스트"] = 공법리스트[:10]  # 최대 10개만 저장
#endregion

안전계획서1_dic["비고"] = ""  # 비고 초기화

hwp.Cancel()

#endregion

#################################################################

#region 섹션명추출
def 섹션명_추출(hwp, save_temp_path="템플릿\\Temp.hwp"):
    """
    HWP 문서에서 섹션 타이틀 추출 및 정리 작업 수행.
    
    :param hwp: 한글(HWP) 객체
    :param save_temp_path: 임시 저장 경로 (기본값: "템플릿\\Temp.hwp")
    """
    # 1. 템플릿 파일 저장
    hwp.SaveAs(save_temp_path, arg="lock:false")
    hwp3 = Hwp(new= True, visible=True)
    hwp3.open(save_temp_path)
    # 2. 모든 표 삭제
    for ctrl in reversed(hwp3.ctrl_list):
        if ctrl.UserDesc == "표":  # 컨트롤이 표일 경우
            hwp3.delete_ctrl(ctrl)  # 바로 삭제

    # 3. 문서의 공백 및 불필요한 내용 정리
    hwp3.MoveDocBegin()
    while hwp3.MoveSelRight():
        selected_text = hwp3.get_selected_text()
        if not selected_text.strip():  # 공백 문자열 삭제
            hwp3.Delete()
        elif hwp3.MoveNextParaBegin():
            continue
        else:
            break
    
    # 4. 강제쪽나눔 전체 삭제
    def delete_forced_page_breaks():
        pset = hwp3.HParameterSet.HGotoE
        hwp3.HAction.GetDefault("Goto", pset.HSet)
        while True:
            try:
                pset.HSet.SetItem("DialogResult", 54)  # 강제쪽나눔으로 이동
                pset.SetSelectionIndex = 5
                if not hwp3.HAction.Execute("Goto", pset.HSet):  # 이동 실패 시 종료
                    break
                hwp3.DeleteBack()  # 강제쪽나눔 삭제
            except Exception as e:
                print(f"오류 발생: {e}")
                break
    hwp3.MoveDocBegin()
    delete_forced_page_breaks()

    # 5. 텍스트 스캔 및 섹션 타이틀 추출
    hwp3.MoveDocBegin()
    hwp3.init_scan()
    extracted_texts = []  # 추출된 텍스트 저장
    while True:
        state, text = hwp3.get_text()
        if text and text.strip():  # 공백 제외
            clean_text = text.replace("\r\n", "").replace("\n", "").replace("\r", "")
            print(clean_text)  # 정리된 텍스트 출력
            extracted_texts.append(clean_text)
        if state <= 1:  # 종료 조건
            break
    hwp3.release_scan()

    # 6. 결과 반환 (추출된 텍스트 리스트)
    return extracted_texts

# hwp 객체를 가져와서 함수 호출
추출한_섹션명_리스트 = 섹션명_추출(hwp)
print(추출한_섹션명_리스트)
# 추출된 텍스트 확인
for title in 추출한_섹션명_리스트:
    print(title)

#endregion

#################################################################



C:\Users\codek\Desktop\건설자료\01. [변경] 안전관리계획서 제1편 - 천안 성성5지구 IPARK 신축공사(R4) 정기안전점검 수정 240925.hwp 01. [변경] 안전관리계획서 제1편 - 천안 성성5지구 IPARK 신축공사(R4) 정기안전점검 수정 240925.hwp
2024. 09
목     차 
목     차 
1. 공사 개요서
2. 현장 위치도
2-1. 현장 위치도
2-2. 현장 주변현황
1) 현장 주변현황
3. 전체 공정표
■ 공사 예정공정표 - 후면 첨부참조
4. 공사 설계도면
■ 건축/구조/토목 설계도서 - 별도 첨부참조
5. 건설물ㆍ공사용 기계설비등의 배치를 나타내는 도면서류
5-1. 가설구조물 등의 설치 및 해체계획
■ 공사용 가설물 배치 계획도 - 후면 첨부참조
5-2. 공종별 장비, 기계 투입계획
1) 공통공사
2) 굴착 및 흙막이지보공
3) 건축(구조물) 공사
4) 건축(마감) 공사
1. 현장 여건 분석
1-1. 지형 및 지질 분석 [소규모 지하안전영향평가서 발췌]
1) 지형분석
2) 지질분석
3) 구지형 분석
(1) 구지형도 분석
(2) 위성사진에 의한 지형분석
1-2. 시추조사 [소규모 지하안전영향평가서 발췌]
1) 시추조사 결과
(1) 지층분포현황
(2) 시추조사에 의한 지층분석 결과
(3) 지층분포 현황
(4) 지층단면도 및 분포특성
(5) 지층별 분포특성 분석결과
2) 현장시험결과 분석
(1) 표준관입시험 결과
(2) 공내 지하수위 측정결과
① 지하수위 측정 결과
② 설계 지하수위
③ 현장 투수시험 결과
④ 현장 암반수압시험 결과
⑤ 공내전단시험 결과
⑥ 공내재하시험 결과(P.M.T)
3) 설계지하수위
(1) 강우에 따른 지하수위 상승고 산정
① 연도별 일 최대 강수량(천안기상관측소, 단위 : mm)
② 일 최대 강우량 발생 후 30일간 강우량(1995년, 천안기상관측소, 단위 : mm)
③ 지표면 조건에 따른 강우 침투율
▌검토단면 A-A
▌A-A 단면 침투해석 결과
▌검토단면 B-B
▌B-B 단면 침투해

In [ ]:
#region 문서내 포지션 추출
#처음위치추가
처음지점추가()

마지막지점추가()

문단시작지점추가()

#문서의 내용 리스트에 추가
#findforward로 섹션리스트를 순회하면서 위치 추가
def 중간위치추가(sec_list):
    hwp.MoveDocBegin()
    for i in sec_list:
        print(i)
        if (hwp.find_forward(i)):
            위치추가()
        else : print(f"섹션{i}를 찾을수 없음.")
중간위치추가(추출한_섹션명_리스트)
# print(cursor_positions, section_titles)

# #para 크기 순으로 재정렬
정렬된_cursor_positions, 정렬된_section_titles = 재정렬_파라기준(cursor_positions, section_titles)
# print(정렬된_cursor_positions, 정렬된_section_titles)

#endregion

#################################################################
 


In [29]:
#region db에 문서 탐색 항목들 입력
insert_data('nnn_1', 'con_1',안전계획서1_dic            )


#endregion

#################################################################



con_1 테이블에 데이터가 삽입되었습니다.
데이터가 없습니다.


In [36]:
print(안전계획서1_dic)

{'파일이름': '01. [변경] 안전관리계획서 제1편 - 천안 성성5지구 IPARK 신축공사(R4) 정기안전점검 수정 240925.hwp', '현장명': '천안 성성5지구 IPARK 신축공사', '현장소재지': '충청남도 천안시 서북구 성성동 34-4 일원', '공사금액': '237,846,000,000원\r\n[VAT별도]', '보고서_날짜_년': '2024', '보고서_날짜_월': '09', '시공자': '에이치디씨현대산업개발(주)', '설계자': '(주)건축사사무소성진', '공사개요_대상공사': '공동주택(아파트) 및 부대복리시설', '공사개요_구조': '철근콘크리트', '공사개요_개소': '11개동', '공사개요_층수지하': '2', '공사개요_층수지상': '33', '공사개요_굴착깊이': '(-)11.32', '공사개요_최고높이': '104.8', '공사개요_연면적': '208,965.63', '기타특수구조물개요': ' -', '주요공법1': '', '주요공법2': '', '주요공법3': '', '주요공법4': '', '주요공법5': '', '주요공법6': '', '주요공법7': '', '주요공법8': '', '주요공법9': '', '주요공법10': '', '파일경로': 'C:\\Users\\codek\\Desktop\\건설자료\\01. [변경] 안전관리계획서 제1편 - 천안 성성5지구 IPARK 신축공사(R4) 정기안전점검 수정 240925.hwp', '비고': '', '주요공법리스트': ['▸ 건 축 - 철근콘크리트구조', '▸ 토 목 - 벽체공법 : C.I.P 공법', '- 지보공법 : STRUT 공법, 제거식 EARTH ANCHOR 공법', '- 차수공법 : S.G.R GROUTING']}


In [37]:
print(정렬된_cursor_positions)

[(0, 0, 16), (500, 0, 11), (2874, 0, 38), (3059, 0, 9), (7464, 0, 17), (0, 14, 8), (0, 47, 9), (0, 47, 9), (0, 54, 8), (0, 64, 9), (0, 67, 9), (0, 68, 12), (0, 71, 13), (0, 72, 12), (0, 78, 9), (0, 79, 22), (0, 80, 10), (0, 81, 27), (0, 82, 31), (0, 83, 24), (0, 85, 28), (0, 86, 12), (0, 87, 9), (0, 90, 16), (0, 93, 15), (0, 95, 14), (0, 99, 0), (0, 111, 9), (0, 115, 9), (0, 123, 11), (0, 124, 13), (0, 131, 19), (0, 142, 12), (0, 145, 12), (0, 147, 22), (0, 155, 13), (0, 157, 18), (0, 171, 19), (0, 173, 14), (0, 174, 15), (0, 182, 18), (0, 185, 15), (0, 187, 12), (0, 190, 15), (0, 194, 17), (0, 197, 14), (0, 204, 21), (0, 211, 11), (0, 214, 24), (0, 218, 25), (0, 218, 35), (0, 226, 22), (0, 230, 12), (0, 233, 18), (0, 237, 12), (0, 240, 18), (0, 244, 12), (0, 247, 18), (0, 251, 21), (0, 253, 16), (0, 256, 24), (0, 259, 14), (0, 262, 9), (0, 264, 11), (0, 267, 26), (0, 270, 47), (0, 272, 52), (0, 278, 9), (0, 281, 26), (0, 283, 11), (0, 287, 20), (0, 289, 20), (0, 292, 7), (0, 295, 15),

In [38]:
print(정렬된_section_titles)

['문서처음', '1. 현장 여건 분석', ' 장비/차량의 통행에 대한 유의사항을 공사 관계자에게 주지 시켰는가', '1. 안전관리조직', '1. 안전사고 발생 및 처리보고', '2024. 09', '목     차 ', '목     차 ', '문단1', '1. 공사 개요서', '2. 현장 위치도', '2-1. 현장 위치도', '2-2. 현장 주변현황', '1) 현장 주변현황', '3. 전체 공정표', '■ 공사 예정공정표 - 후면 첨부참조', '4. 공사 설계도면', '■ 건축/구조/토목 설계도서 - 별도 첨부참조', '5. 건설물ㆍ공사용 기계설비등의 배치를 나타내는 도면서류', '5-1. 가설구조물 등의 설치 및 해체계획', '■ 공사용 가설물 배치 계획도 - 후면 첨부참조', '5-2. 공종별 장비, 기계 투입계획', '1) 공통공사', '2) 굴착 및 흙막이지보공', '3) 건축(구조물) 공사', '4) 건축(마감) 공사', '문단2', '1) 지형분석', '2) 지질분석', '3) 구지형 분석', '(1) 구지형도 분석', '(2) 위성사진에 의한 지형분석', '1) 시추조사 결과', '(1) 지층분포현황', '(2) 시추조사에 의한 지층분석 결과', '(3) 지층분포 현황', '(4) 지층단면도 및 분포특성', '(5) 지층별 분포특성 분석결과', '2) 현장시험결과 분석', '(1) 표준관입시험 결과', '(2) 공내 지하수위 측정결과', '① 지하수위 측정 결과', '② 설계 지하수위', '③ 현장 투수시험 결과', '④ 현장 암반수압시험 결과', '⑤ 공내전단시험 결과', '⑥ 공내재하시험 결과(P.M.T)', '3) 설계지하수위', '(1) 강우에 따른 지하수위 상승고 산정', '① 연도별 일 최대 강수량(천안기상관측소, 단위 : mm)', ' 단위 : mm)', '③ 지표면 조건에 따른 강우 침투율', '▌검토단면 A-A', '▌A-A 단면 침투해석 결과', '▌검토단면 B-B', '▌B-B 단면 침투해석 결과', '▌검토단면 C-C', '▌

In [35]:
print_data('nnn_1', 'con_1',)

데이터가 없습니다.


In [ ]:
#region db에서 custom_data 특정 id만 추출
selected_ids = [2, 4,7]  # 사용자가 선택한 ID
#custom_data = fetch_custom_data_with_next_positions(selected_ids)
#print(custom_data)
#endregion

#################################################################



In [ ]:
#region 복붙하기 

#내용 삽입 함수
def 복붙_내용삽입(custom_data):
    """
    custom_data: [(섹션시작좌표, 섹션끝좌표, 섹션명), ...] 형태의 리스트
    """
    for section_start, section_end, section_name in custom_data:
        # 섹션 시작 위치로 이동
        hwp.SetPos(*section_start)  
        hwp.MoveNextParaBegin()  # 다음 단락으로 이동
        hwp.Select()  # 섹션 시작 위치에서 선택 시작

        # 섹션 끝 위치로 이동
        if section_end == None:
            print("잘못된 id를 추출하였습니다.(문서끝id입력함)")
        else:
            hwp.SetPos(*section_end)
            hwp.MoveLeft()  # 끝 위치에서 한 글자 왼쪽으로 이동해 선택 범위 조정

        # 복사 작업 수행
        hwp.Copy()

        # 두 번째 HWP 파일의 필드로 이동하여 붙여넣기
        #hwp2.MoveToField(section_name)
        #hwp2.Paste()  # 복사한 내용 붙여넣기

#복붙_내용삽입(custom_data)
#endregion

#################################################################


In [3]:
print(추출한_섹션명_리스트)

['2024. 09', '목     차 ', '목     차 ', '1. 공사 개요서', '2. 현장 위치도', '2-1. 현장 위치도', '2-2. 현장 주변현황', '1) 현장 주변현황', '3. 전체 공정표', '■ 공사 예정공정표 - 후면 첨부참조', '4. 공사 설계도면', '■ 건축/구조/토목 설계도서 - 별도 첨부참조', '5. 건설물ㆍ공사용 기계설비등의 배치를 나타내는 도면서류', '5-1. 가설구조물 등의 설치 및 해체계획', '■ 공사용 가설물 배치 계획도 - 후면 첨부참조', '5-2. 공종별 장비, 기계 투입계획', '1) 공통공사', '2) 굴착 및 흙막이지보공', '3) 건축(구조물) 공사', '4) 건축(마감) 공사', '1. 현장 여건 분석', '1-1. 지형 및 지질 분석 [소규모 지하안전영향평가서 발췌]', '1) 지형분석', '2) 지질분석', '3) 구지형 분석', '(1) 구지형도 분석', '(2) 위성사진에 의한 지형분석', '1-2. 시추조사 [소규모 지하안전영향평가서 발췌]', '1) 시추조사 결과', '(1) 지층분포현황', '(2) 시추조사에 의한 지층분석 결과', '(3) 지층분포 현황', '(4) 지층단면도 및 분포특성', '(5) 지층별 분포특성 분석결과', '2) 현장시험결과 분석', '(1) 표준관입시험 결과', '(2) 공내 지하수위 측정결과', '① 지하수위 측정 결과', '② 설계 지하수위', '③ 현장 투수시험 결과', '④ 현장 암반수압시험 결과', '⑤ 공내전단시험 결과', '⑥ 공내재하시험 결과(P.M.T)', '3) 설계지하수위', '(1) 강우에 따른 지하수위 상승고 산정', '① 연도별 일 최대 강수량(천안기상관측소, 단위 : mm)', '② 일 최대 강우량 발생 후 30일간 강우량(1995년, 천안기상관측소, 단위 : mm)', '③ 지표면 조건에 따른 강우 침투율', '▌검토단면 A-A', '▌A-A 단면 침투해석 결과', '▌검토단면 B-B', '▌B-B 단면 침투해석 결과', '▌

In [32]:
#region db에서 custom_data 특정 id만 추출
selected_ids = [55]  # 사용자가 선택한 ID
custom_data = fetch_custom_data_with_next_positions(selected_ids)
print(custom_data)
#endregion

[((0, 237, 12), (0, 240, 18), '▌검토단면 B-B')]


In [ ]:
reset_table()

In [6]:
#필드추가
text_list = ["가나다1", "가나다2", "가나다3", "가나다4"]
for i in 추출한_섹션명_리스트:
    if hwp.find_forward(i):
        hwp.MoveParaEnd()
        hwp.BreakPara()
        hwp.create_field(name=i, direction=i)

In [ ]:
#db에서 custom_data 특정 id만 추출
selected_ids = [2, 4,7]  # 사용자가 선택한 ID
custom_data = fetch_custom_data_with_next_positions(selected_ids)
print(custom_data)

In [86]:
custom_data.clear()


In [ ]:
# 데이터 업데이트
update_data(3, 비고="수정된 비고")
# 데이터 조회 및 출력
print_data()

In [ ]:
# 데이터 삭제
delete_data(2)
# 데이터 조회 및 출력
print_data()



In [154]:
#####복붙 TEST 용#####

#내용 삽입 함수
def 복붙_내용삽입(custom_data):
    """
    custom_data: [(섹션시작좌표, 섹션끝좌표, 섹션명), ...] 형태의 리스트
    """
    for section_start, section_end, section_name in custom_data:
        # 섹션 시작 위치로 이동
        hwp.SetPos(*section_start)  
        hwp.MoveNextParaBegin()  # 다음 단락으로 이동
        hwp.Select()  # 섹션 시작 위치에서 선택 시작

        # 섹션 끝 위치로 이동
        if section_end == None:
            print("잘못된 id를 추출하였습니다.(문서끝id입력함)")
        else:
            hwp.SetPos(*section_end)
            hwp.MoveLeft()  # 끝 위치에서 한 글자 왼쪽으로 이동해 선택 범위 조정

        # 복사 작업 수행
        hwp.Copy()

        # 두 번째 HWP 파일의 필드로 이동하여 붙여넣기
        hwp2.MoveToField(section_name)
        hwp2.Paste()  # 복사한 내용 붙여넣기

복붙_내용삽입(custom_data)